# Level UP your Python

### Henry Schreiner

In [ ]:
%config InteractiveShell.ast_node_interactivity="last_expr_or_assign"

# Could enable pretty tracebacks (ONLY DO IN APPLICATION, NOT LIBRARY):
# import rich.traceback
# rich.traceback.install(show_locals=True)
#
# Or pretty-printing by default
# rich.pretty.install()

# Intro

### Expected Knoledge

You should already know:

* Basic Python syntax
* Functions
* Basic classes (will cover quickly)
* Basic NumPy (will mention)
* Git - _CRITICAL FOR ANY SOFTWARE WORK!_

And we will be using notebooks in JupyterLab, but won't be talking about them much, except for one disclaimer: They are for teaching, testing, prototyping, and final analysis driving - they are _NOT_ a place to store significant amounts of code! Refactor into a file once you have developed something!

### What can you expect?

* I don't know what you know
* We don't have time to study any topic in depth.

So, we will move _fast_, and cover a _lot_.

You are not expected to able to master everything you see.

### Instead, you are expected to:

1. Know what is possible, so you know to look for it
2. Get pointers on where to look (lots of links!)
3. Refer back to this material later.

## About the author

[![Henryiii's github stats](https://github-readme-stats.vercel.app/api?username=henryiii)](https://github.com/anuraghazra/github-readme-stats)

https://iscinumpy.gitlab.io

Scikit-HEP admin, member of IRIS-HEP.

### Projects

cibuildwheel • pybind11 • boost-histogram • Hist • Vector • CLI11 • Plumbum • GooFit • Particle • DecayLanguage • Conda-Forge ROOT • POVM

### Intrests:

Packaging and building • Bindings • Building a HEP analysis toolchain in Python, JITable

## Theory: New features in programming

Programming is all about _orginization_. This is not always obvious, and has some odd consequences. Let's look at one: **new features remove functionality from the user**.

Don't believe me? Pick one. Let's go with an old, simple one you should already know: `goto` vs. loops (`for`/`while`). (This is my favorite example, even though Python thankfully came along late enough to not even have `goto` in the first place, except as an April fools joke.)


You have total power with goto! Jump from anywhere, too anywhere! You can recreate all loops with it, and more! So why are loops the newer, better feature?

**goto ([partially hypothetical](https://github.com/snoack/python-goto) in Python)**
```python
i = 0
label .start
print(f"Hi {i}")
i + 1
if i <= 10:
    goto .start
```

**Compare to for loop:**
    

In [ ]:
for i in range(10):
    print(f"Hi {i}")

A programmer has to spend time to recognize what is happing in the first example - in the second example, even a fairly new Python programmer will immediatly say "that prints 0 to 9". The second example lets you build more complex programs, because you are working at a 'higher level', humans tend to do better which high level concepts (while computers work up from low level).

# Python Object Model

## Everything in Python is an object!

> Okay, technically a `PyObject*` in CPython - we'll focus on CPython most of the time today.

An object is an "instance" of a "type", or a "class", which describes what that sort of object does.

> Types and basic objects have some optimizations in CPython, for speed and also to keep the language from being infinitely recursive - but they are still PyObject*'s.
>
> Also, "built-in" classes are a little special.

In [ ]:
v = 4

In [ ]:
v.bit_length()

In [ ]:
def f(x: int) -> int:
    return x**2

In [ ]:
f(4)

This really is an assignment, a new fuction is made and assigned to `f`. But, since the assignment is not arbitrary, that is, it clearly has a name, functions remember their name!

In [ ]:
f.__name__

In [ ]:
f, v = v, f
# What is the v now? f? How about the name?

## Inspecting

You can inspect objects. There are lots of ways.

* In a juptyer notebook, use `object.<tab>` to bring up completions, shift tab for help.
* You can use dir(object) to see all attribututes (more or less)
* You can use help(object) or object? (IPython only) to see help
* You can `import inspect` and use the tools there
* You can install the rich library and use `rich.inspect()`

In [ ]:
def f(x: float) -> float:
    "I am a square!"
    return x**2

help(f)

In [ ]:
dir(f)

In [ ]:
import inspect
print(inspect.getsourcefile(f))
print(inspect.getsource(f))
# WARNING! THIS DOES NOT ALWAYS WORK!

**WARNING: You cannot *always* see the source of a function, so this is a user trick, not one to use in a library!**

Python does a three stage procedure when interpreting. It convers source to bytecode (pyc files), then runs the bytecode in the interpreter. When loading a file that has been run before (or came from a wheel, more on that later), it only loads the bytecode if the source hasn't changed - the source is not reparsed. So inspect works by looking up the original file location. _But you can delete the original file and run from bytecode only!_. Don't do that, but you can. Also, you can run from a zip file, and the orignal file might not be openable. And, finally, when running live in a REPL, there may not be a source (it works in IPython for us, though).


In [ ]:
import rich
rich.inspect(f)

In [ ]:
rich.inspect(3)

## Mutablity

Many built-in objects are immutable, so it can hide the fact that Python does not have the concept of "pass by value"; the labels you see really are all pointing to PyObject*'s that are being managed by Python's garbage colllector.

If you write `x = y`, then x and y refer to the same object. Always. This can't be overriden. However, it's hard to see that if you can't mutate the value, so there's no "side effect" to this. Side effects only happen for mutable objects.

In [ ]:
x = 3
y = x
x += 1
print("What is {y}?")

Now, let's try a mutable object. Lists, sets, and dicts are the most notable mutable objects in `builtins`.

In [ ]:
x = [3]
y = x
x[0] += 1
print("What is {y = }?")

Why?

The problem was that when the object was imuatable, the inplace operator `+=` actually behaved like `x = x + 1`, which is a new object. When it was a mutable object (a list), then it was able to change it in-place.

#### Advanced aside: Why did this work?

Quick aside for advanced Pythonistas, this is tricky. `x[0]` returns an int. So why is this any different than before? Let's explore, using mock:

In [ ]:
from unittest.mock import MagicMock

ListProxy = MagicMock("ListProxy")
y = ListProxy()
y[0] += 1
rich.print(y.mock_calls)

You can see that this has special support for this syntax, it pulls out the item inside, and sets it, then stores it. There are other special syntax treatments in Python as well, all designed to make the language more friendly and powerful:



In [ ]:
1 < 2 < 3

In [ ]:
print(not 3 in {1, 2, 3})
print(3 not in {1, 2, 3})

Can we glean anything from the original example, though? Yes, assignment is special. There are only three forms of normal assignment:

```python
x = y = 1
x[y] = 2
x.y = 3
```

That's it. These are _not_ valid assignments:

```python
x(y) = 1    # There is no assignment for __call__
x + y = 2   # Arbitrary expressions are not allowed
(x = 2) + 2 # Can't be nested
# So on.
```

> This was limiting, so Python 3.8 added a nestable assignment, `:=` (walrus operator), though it was somewhat controversial. It does not work anywhere that normal `=` works, to avoid confusion.

Inplace assignment operators follow the same rules, so they are "special", and not allowed anywhere.

## Scope

Python has scope, but not in very many places. Functions and class definitions create scope, and modules have scope. (Generator expressions now have scope too). That's about it. So you can write this:

In [ ]:
if True:
    x = 3
print(x)

If `if` had scope, then x would not be accessable outside the if. This is simple and useful, but you have to be careful to stay clean and tidy. For example, if that `if` was `False`, this would suddenly break. It's valid to use the loop varaible after a loop ends, ect. There's really not much scope at all!

Because it shows up in so few places, and is so close to automatic, in can bite you once in a while if you don't keep it in mind:

In [ ]:
x = 1
def f() -> None:
    print(x)
    
f()
print(x)

But we will try an assignment:

In [ ]:
x = 1
def f() -> None:
    x = 2
    print(x)
    
f()
print(x)

So x in the function is not the same x out of the function now! (Try printing before assigning to it, or try changing it inplace. Even better, what happens if it is mutable?)

If you want a rule to put in your pocket:
* Accessing a variable uses the first variable it finds going up in scope
* Setting a variable always uses the local scope

If this is really what you want, you can use `nonlocal` to access a variable one-scope-up but not global, or `global` to declare a variable with global scope.

In [ ]:
x = 1
def f() -> None:
    global x
    x = 2
    print(x)
    
f()
print(x)

Need a pratical rule?

**Always pass variables out explicilty, be cautious with using anything not clearly global in a function.**

This means you should never see `global`, as it's only needed for setting variables. Global read-only variables (the only safe kind) are often ALL_CAPS.

## Memory and the Garbage Collector

Python is a garbage collected language. All Python objects have a refcount, which tells the garbage collector how many "labels" or how many ways you can access that object. When you can't get to the object anymore, the garbage collector _has the right to_ remove the object. It _usually_, depending on settings, runs rougly once per line, but it doesn't have to.

As a consequence, _never_ depend on an object being deleted to perform some action at some time. More on that later, when we cover context managers.

Let's look at it:

In [ ]:
import gc
import sys

class Boom:
    def __del__(self) -> None:
        print("Boom!")

ob = Boom();

In [ ]:
gc.is_tracked(ob)

In [ ]:
sys.getrefcount(ob)

In [ ]:
del ob

If all went well, you probably should have seen "Boom" above. Now let's try a variation:

In [ ]:
ob = Boom()
ob

In [ ]:
del ob

Notice anything? Probably not. Objects only get deleted when the refcount goes to 0 (okay, technically 1, since the GC holds a refernce to it too; otherwise it wouldn't know what to delete. So it goes to 0 as it gets deleted by the gc). IPython tracks outputs; you can access all of them:

In [ ]:
Out[17]

There it is! Since we can access it, it's not collectable. (In fact, we can now access it from two Out's! )


That's just one case where an object doesn't get garbaged collected until the final cleanup. You can also turn the garbage collector off, it could be running at a different setting, etc.

Also, during the cleanup phase at the end, everything starts getting deleted. So even _modules_ might not be safe to call in cleanups. You have been warned.

Don't use `__del__` in almost all cases, except for emergancy cleanup. There is a better way if you want "action at a distance", as I like to call it; we will see it when we get to context managers.

## Importing

Caveat: A few minor details below will not be quite right for namespace packages, which do not contain an `__init__.py`. They are not heavily used, and designed for a specific purpose which we won't cover. And the only change is I'd have to be much more wordy but would still basically be saying the same things.

### Importing Basics


```python
import a.b       # line 1
from a import b  # line 2
```

So, question: is `b` a module (`b.py`), a package (`b/__init__.py`), or an object if line 1 is valid? How about 2?

The rule: The left part must be a package. The right part of a `from ... import ...` statement can be anything.

Any packages that appear in this statement have their `__init__.py` run unconditionally.

So, a second question: is this valid?

```python
import a
a.b
```

If `b` is a package or module, this only works it imported in `__init__.py`.

### Where do imports come from?

If I `import a`, where is `a`? Python works down sys.path:

* `sys.path` _starts with_ `"."` - be careful about using this, and it _does_ override system site packages
* Then the standard library follows
* Then the system site packages and user site packages

So a package containing `package/sys.py` is fine, even `package/__init__.py` will be able to import the normal `sys`, and has to import `package.sys` or `.sys` to get the local `sys.py`.

> You may see `from __future__ import absolute_import`. This was from Python 2, causing the import system to work like Python 3 and prioritize the system site packages over local packages. With the default behavior, Python 2 could not access the system modules if a local module had the same name!

### Relative imports

```python
from .sys import my_function
```
These can be moved around, but have some drawbacks. They only work in the syntax above (`from`), they only work when running _as_ a package (so a `__init__.py`), you can't use them in a non-library `__name__ == "__main__"` file that you direclty run (but `python -m ...` runs are fine).

# Object Oriented Programming

Objects are _very_ powerful, and you can harness this power for your own designs!

## Writing a class

Writing a class should be second nature, and very natural to you - just as natural as writing a function!

In [ ]:
def f(x: float) -> float:
    return x**2

print(f"{f(3) = }")

In [ ]:
class F:
    def __call__(self, x: float) -> float:
        return x**2
    
f = F()
print(f"{f(3) = }")



#### Data + functions

They allow you to bundle data with the functions that run on them. In a language like Python without (much) type overloading, this is important for design (and good for tab completion).

In [ ]:
# Built in for Python 3.7+, install otherwise
from dataclasses import dataclass

@dataclass
class Vector:
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y
        
    def mag(self) -> float:
        return (self.x**2 + self.y**2)**.5

> Note: developing a class that represents data has a bit of boilerplate involved, so attrs (third-part) or dataclasses (first party in Python 3.7, thirdparty for 3.6) has a trick to make them _much more easily_, and automatically add things like nice repr's. We'll cover the parts of the syntax here later.

In [ ]:
# Built in for Python 3.7+, install otherwise
from dataclasses import dataclass

@dataclass
class Vector:
    x: float
    y: float
        
    def mag(self) -> float:
        return (self.x**2 + self.y**2)**.5

In [ ]:
v = Vector(3,4)
v.mag()

#### Functors

I just told you it was a bad idea to set something outside your local scope, and often not even a good idea to just view something outside the local scope. So how do you write something that has scope? Use a class as a functor! Compare this:

In [ ]:
_start = 0
def incr() -> int:
    global _start
    _start += 1
    return _start

In [ ]:
incr()

In [ ]:
incr()

With this:

In [ ]:
class Incr:
    def __init__(self, incr: int = 0) -> None:
        self.incr = incr
    def __call__(self) -> int:
        self.incr += 1
        return self.incr

In [ ]:
incr = Incr()
incr()

In [ ]:
incr()

This is explicit, clear, I can have multiple instances without having them interfere, I can see exactly what's going on without having to trace down a global, and I can even set the default value when I make the Incr instance!

#### Modularity

Let's say you have an algorithm with three parts. If you make a class that calls three member functions, you can allow a user to replace just one of the functions, and use the original first two. Classes are great for _originization_ and _code reuse_ because of this.

In [ ]:
class RunSomethingHard:
    def part1(self) -> None:
        print("Working hard")
    def part2(self) -> None:
        print("Working harder")
    def part3(self) -> None:
        print("That was hard!")
    def run(self) -> None:
        self.part1()
        self.part2()
        self.part3()

In [ ]:
inst = RunSomethingHard()
inst.run()

Now, look at how I can swap out part of the calculation without rewriting from scratch!

In [ ]:
class NewRunSomethingHard(RunSomethingHard):
    def part2(self) -> None:
        print("Nah, this is easy")

In [ ]:
inst = NewRunSomethingHard()
inst.run()

#### DSL (Domain Specific Language)

You can customize almost every behavoir of a class to make them very natural for whatever you are doing.

In [ ]:
class Path(str):
    def __truediv__(self, other):
        return self.__class__(f"{self}/{other}")

In [ ]:
Path(".") / "myfile" / "program.py"

> Just in case you want to make a Path class like the one above - don't, use pathlib instead. We could ahve written `self.__class__` as `Path`, but then this would not subclass correctly and besides, using the class name inside the class is ugly and makes it harder to rename. If you return a normal string, then you can't keep applying `/`.
>
> Also, I left off type annoations for this example, as to do them properly I need to use a TypeVar.

#### Mixins (advanced)

If you follow good practices, you can even make collections of behaviors and mix them into other classes - specifically, a mixin should not have an `__init__` or any new datamembers. (The second requirement is more important than the first, if you are carful to use `super()`. Let's rewrite the last example with mixins:

In [ ]:
class PathMixin:
    def __truediv__(self, other):
        return self.__class__(f"{self}/{other}")
    
class Path(str, PathMixin):
    pass

Path(".") / "myfile" / "program.py"

> Potentially show demo here?

#### Other: ABC, Protocols, and more

Other useful things to look into are ABCs (Abstract Base Classes) and Protocols. These let you a) require certain methods be implemented by users, and b) formalize "duck typing". ABC's are a run-time feature, and kind of half-broken; you have to instanciate an ABC class to get the benefit of the checking. Protocols are a "type-check time" feature, and are far better and don't require special inheratince, but only are enforced by type checkers (see a later section!).

## Design considerations

Object oriented programming has been known to make it easy to create spegetti messes of code. The following tips will help you not fall into the trap and end up with poorly designed code. "Make it a class", by itself, will not magically make your code better.

#### Modular design

You should break your code into _concepts_, and classes should help map those concepts to the computer. Different components of a detector might be classes, with an instance for each component. A vector, a URL, a remote data source, etc. You might have a class representing a unit of an analysis, and use either inheritance (okay) or a protocol (better, reduces coupling) to have real data processing vs. simulation generation. Etc.

#### Unit test

We will mention testing later, and there is another course on it, but I'm focusing on the word _unit_. You need to be able to run your classes standalone, in unit tests, and not only inplace. This keeps the design modular - you will resist the desire to make a class that needs a class to make another class inside a class that only works with the file that sits on your work laptop, etc. And you'll be free to redesign parts without having to worry about everything breaking down.

Always use PyTest for unit testing.

# Part 2: Control flow and Features

# Errors and catching them

There are two types of errors in Python. A few really bad erros are segfaults. These are almost always something triggered via the C interace (such as by using `ctypes`), and are not due to problems your Python code. _Most_ errors in Python are part of the langauge, called Exceptions.

An Exception is just a special control flow feature for things that are "exceptional"; often errors, but they are used for other things. In fact, internally, loops end by triggering a special exception!

Exceptions "bubble up" through the stack to the outermost scope. If you catch an exception before it reaches the top, you can handle it. If you don't, then it shows up on the screen or in your logs as a traceback.

In [ ]:
# Uncomment this to see an exception:
# 1 / 0

In [ ]:
try:
    1 / 0
except ZeroDivisionError:
    pass

Exceptions use inhertance to form a tree structure, so you can be as tight or as loose as needed in catching them. Let's see the parents of Zero Division error:

In [ ]:
ZeroDivisionError.__mro__

You could catch any of these instead:

In [ ]:
try:
    1 / 0
except ArithmeticError as e:
    print(repr(e))

Always catch the most narrow exception you can! Never try to catch a really broad class or all exceptions, because things like running out of memory, exit signals, and more are exceptions too, and you don't want to catch those if you didn't mean to!

Here's a basic example of making your own:

In [ ]:
class MyNewException(RuntimeError):
    pass

try:
    raise MyNewException()
except MyNewException:
    pass

There can be as many `except` blocks as you need, and there's also a `finally` block, which will always run, even if the exception is caught:

In [ ]:
try:
    1 / 0
except ArithmeticError:
    print("Caught the exception!")
finally:
    print("I can run cleanup, regardless of what happens above!")

Where would you want something like this? How about closing a file!

```python
try:
    f = open(...)
    # do stuff with f that might throw an exception (basically anything)
finally:
    f.close()
```

This way, if an exception is thrown, the file still gets nicely closed. In fact, this is so important, we'll see a feature built around it soon!

# Generators (Iterators)

Let's change topics just for a moment (we'll get to context managers in a moment, which we are building toward). Let's look into iterators, which are a form of generator. I'm sure you've seen one:

In [ ]:
for i in range(4):
    print(i)

But what is `range(4)`? It's not a list, it's a custom object:

In [ ]:
range(4)

Python has built into it the concept of iteration. What the various looping structures do is call `iter()` on the object first, then call `next()` over and over until a `StopIteration` Exception is raised. Try it:

In [ ]:
it = iter(range(0, 4))

In [ ]:
next(it)

You could implement `__iter__` and `__next__` yourself, but Python has a built in syntax shortcut for making iterators (and generators, which have a `__send__` too): 

In [ ]:
def range4():
    yield 0
    yield 1
    yield 2
    yield 3

In [ ]:
for i in range4():
    print(i)

The presence of a single yield anywhere in a function turns it into an iterator. Notice "calling" the iterator factory function just produces an iteratable object, it does not run anything yet. Then, when you iterate it, it "pauses" at each yield.

Many Python functions take iterables, like `list` and `tuple`:

In [ ]:
list(range4())

If Python were rewritten today, there would likley be a keyword, like `iter def`, to indicate that a `def` is making an iterator instead of a normal function; but for historical reasons, you just have to look for `yield`'s inside the function. 

# Decorators

This is likely the simplest syntactic sugar you'll see today, but maybe one with some of the furthest reaching consequences. Let's say you have a bit of code that looks like this:

```python
def f(): ...
f = g(f)
```

So `g` is a function that takes a function and (hopefully) returns a function, probably a very similar one since you are giving it the same name as the old "f". In Python 2.5, we gained the ability to write this instead:

```python
@g
def f(): ...
```

That's it. The thing after the `@` "decorates" (or transforms) the function you are defining and the output is saved with the name `f`.

In [ ]:
def bad_decorator(func):
    print(f"You don't need {func.__name__}!")
    return 2

@bad_decorator
def f(x):
    return x**2

In [ ]:
f

Okay, so that's useless (well, except for the printout, which could be good for logging). What could this be used for? Turns out, almost anything. Having a syntax for "modifying" a function (it also works on methods and classes, too) is fantastic, and lets you think in a different way.

There are several decorators in builtins, `property`, `classmethod`, and `staticmethod`. For example:

In [ ]:
class BagOfFunctions:
    @staticmethod
    def f(x):
        return f**2

What's missing from the above function? Self! It's static, it doesn't need an instance, or even the current class.

In [ ]:
BagOfFunctions.f(2)

In [ ]:
BagOfFunctions().f(2)

The decorator took our method and added the correct handling to it so it works with or without an instance.

If the thing after the `@` is called, this is called a decorator factory; it's exactly the same as above, just slightly more unusual in structure to what you normally see:

```python
def f...
f = g()(f)

# same as

@g()
def f...
```

You can also nest decorators.

You could have a rate decorator, which causes a function to wait after completing so that it always takes the same amount of time. You could have a logging decorator, which prints to a log every time the wrapped function is called. There are quite a few decorators in the standard library; we'll see more later, but here are a couple interesting ones:

#### Least Recently Used Caching

This is all you need to implement a cache based on the input arguments. When you call this again with recently used arguments (the cache size is adjustable), it pulls from a cache instead of rerunning the function. (Note: parentheses on this one are optional in Python 3.8, it's both a decorator and a decorator factory starting in 3.8)

In [ ]:
import functools
import time

@functools.lru_cache()
def slow(x: int) -> int:
    time.sleep(2)
    return x

In [ ]:
slow(4)

In [ ]:
slow(4)

Another magical decorator is `functools.singledispatch`, which lets you simulate type based dispatch (but only on the first argument) from other langauges:

In [ ]:
@functools.singledispatch
def square(x):
    print("Not implemented")

@square.register
def square_int(x: int) -> int:
    return x**2

@square.register
def square_str(x: str) -> str:
    return f"{x}²"

In [ ]:
square(2)

In [ ]:
square("x")

There's also `@functools.total_ordering`, which when applied to a class, fills in the missing comparison operators from the ones that are already there (`==`, `!=`, `<`, `<=`, `>`, `>=` can be computed from just two functions)

And `@functools.wraps` is a decorator that helps you write decorators that wrap functions. Also see decorator and the newer, fancier wrapt libraries on PyPI.

Another use case we've breifly seen is dataclasses from Python 3.7:

In [ ]:
from dataclasses import dataclass

@dataclass
class Vector:
    x: float
    y: float

This `@dataclass` is taking the class you pass in, converting the class annotations to instance variables, making an `__init__`, `__repr__`, and much more. When you are viewing a class as data + functionality, this is a very natural way to work.

In [ ]:
Vector(1, y=2)

If you need Dataclasses in 3.6, there's a pip install dataclasses backport, and this was based on the popular `attrs` library, which is much more powerful and can do all sorts of tricks, like validate and transform values.

`click` is a package that lets you write command line interfaces using decorators on functions:

```python
import click

@click.command()
@click.option('--count', default=1, help='Number of greetings.')
@click.option('--name', prompt='Your name',
              help='The person to greet.')
def hello(count, name):
    """Simple program that greets NAME for a total of COUNT times."""
    for x in range(count):
        click.echo('Hello %s!' % name)

if __name__ == '__main__':
    hello()
```

We'll see more.

# Context Managers

Yes! Our journey is complete, we are where I wanted to be. Context managers are one of my favorites, and a little underused, expecially in user code, when they are really easy to both write and use (while decorators, for comparison, are really easy to use but a bit tricky to write). A context manager has a specific purpose.

The first is what I call "action at a distance". It lets you schedule an action for later that is sure to always happen (unless you get a segfault, or exit a really nasty way). This is likely the most famous context manager:

```python
with open(...) as f:
    txt = f.readlines()
```

When you enter the with block, `__enter__` is called and the result is assigned to the `as` target, if there is one. Then when you leave the block, `__exit__` is called. If you leave via an exception, `__exit__` gets special arguments that let you even decide what to do based on that the exception is - or even handle the exception and continue. `contextlib` has several simple context managers, like `redirect_stdout`, `redirect_stdout`, and `suppress`:

In [ ]:
import contextlib

with contextlib.suppress(ZeroDivisionError):
    1 / 0

But the real star of contextlib is `contextmanager`, which is a decorator that makes writing contextmanagers really easy. Let's try one of my favorites, a timer context manager:

In [ ]:
@contextlib.contextmanager
def timer():
    old_time = time.monotonic()
    try:
        yield
    finally:
        new_time = time.monotonic()
        print(f"Time taken: {new_time - old_time} seconds")

In [ ]:
with timer():
    print("Start")
    time.sleep(1.5)
    print("End")

As an extra bonus, `contextmanager` uses `ContextDecorator`, so the objects it makes can also be used as Dectorators! [Pretty much everything](https://docs.python.org/3/library/contextlib.html) in the `contextlib` module that does not have the word `async` in it is worth learning. `contextlib.closing` turns an object with a `.close()` into a context manager, and `contextlib.ExitStack` lets you nest context managers without eating up massive amounts of whitespace.

In [ ]:
@timer()
def long_function():
    print("Start")
    time.sleep(1.5)
    print("End")
    
long_function()

# Quick note: Async

Everything we've been doing has built on itself, and we seemed to be going somewhere; the pinnicle of this direction was asctually not context managers, but `async/await`. All of this feeds into `async/await`, which was formally interoduced as a langauage component in Python 3.6. However, we did skip two neccesary steps; we didn't talk about generators (iterators can actually "send" values in, not just produce them, but there's no specific constuct for doing that, like there is for consuming values in a for loop), and we didn't talk about `yield from`, which lets you put an iterator or generator inside another one. The main reason we didn't try to reach `async` though is that I've never found a great use for it in scientific programming; it is much more intrusive than normal threading, it doesn't really "live" side-by-side with normal syncronious programming all that well (it's better now, though), and the libraries for it are a little young. Feel free to investigate on your own, though! I've also discussed the mechanisms behind it in detail in my blog.

# Static Type Hinting

The most exciting thing happening right now in Python development is static typing. Since Python 3.0, we've had function annoations, and since 3.6, variable annoations. In 3.5, we got a "typing" library, which provides tools to describe types. You've already seen me using type hints:

In [ ]:
def f(x: int) -> int:
    return x * 5

You might have been asking yourself, what does that do? Does it limit what I can use here?

In [ ]:
f(["hi"])

No. It does nothing at runtime, except store the object. And in the upcoming Python 3.10 (or 3.7+ with `from __future__ import annotations`, it doen't even store the actual object, just the string you type here.

It is not useless though! For one, it helps the reader. Knowing the types expected really gives you a much better idea of what is going on and what you can do and can't do.

But the key goal is: static type checking! There are a collection of static type checkers, the most "offical" and famous of which is MyPy. You can think of this as the "compiler" for compiled languages like C++; it checks to make sure you are not lying about the types. For example:

In [ ]:
%%writefile tmp_mypy1.py
def f(x: int) -> int:
    return x * 5

f(["hi"])

In [ ]:
!mypy tmp_mypy1.py

There we go! And, most importantly, _we didn't have to run any code to see this error_! Your tests cannot test every possible branch, every line of code. MyPy can (though it doesn't by default, due to gradual typing). You may have code that runs rarely, that requires remote resources, that is slow, etc. All those can be checked by MyPy. It also keeps you (too?) truthful in your types.

Let's see an exmaple of an error that MyPy can catch:

In [ ]:
%%writefile tmp_mypy1.py
from typing import Optional

def f(x: Optional[int]) -> Optional[int]:
    return x * 5

f(4)

In [ ]:
!mypy tmp_mypy1.py

Your test suite may have forgotton to run with a None input. You may not run into None often, until you are in a critical situation. But MyPy can find it and tell you there's a logic issue, your function cannot take `None` like it claims it can.

Personally, I recommend using pre-commit to run all your checks except pytest (and that only because it's likely slow), and including mypy in your pre-commit testing. Try to turn on as much as possible, and increase it until you can run with full `strict` checking.

# Packaging and Files

# PyTest: Unit Testing
# NumPy: You should know this one
# Pandas: DataFrames for Python
# Numba: JIT for Speed!
# Pybind11: Use C++ libraries
# Code Quality and CI